## Project File Structure

After the project is complete, We will be left with all these files. they explain how the project will be implemented.

#### Train_chatbot.py: In this file, we will build and train the deep learning model that can classify and identify what the user is asking to the bot.

#### Gui_Chatbot.py: This file is where we will build a graphical user interface to chat with our trained chatbot.

#### Intents.json: The intents file has all the data that we will use to train the model. It contains a collection of tags with their corresponding patterns and responses.

#### Chatbot_model.h5: This is a hierarchical data format file in which we have stored the weights and the architecture of our trained model.

#### Classes.pkl: The pickle file can be used to store all the tag names to classify when we are predicting the message.

#### Words.pkl: The words.pkl pickle file contains all the unique words that are the vocabulary of our model.

Download the source code and the dataset: https://drive.google.com/drive/folders/1r6MrrdE8V0bWBxndGfJxJ4Om62dJ2OMP?usp=sharing



## How to Build The Chatbot
1. Import Libraries and Load the Data
2. Preprocessing the Data
3. Create Training and Testing Data
4. Training the Model
5. Interacting With the Chatbot

### Step 1. Import Libraries and Load the Data
Create a new python file and name it as train_chatbot and then we are going to import all the required modules. After that, we will read the JSON data file in our Python program.

1. numpy (np): NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on these arrays.
2. keras.models.Sequential: Sequential is a type of Keras model for creating neural networks layer-by-layer, where each layer has exactly one input tensor and one output tensor.
3. keras.layers.Dense: Dense is a type of layer in a neural network, where every neuron in the layer is connected to every neuron in the preceding layer.
4. keras.layers.Activation: Activation functions are used to introduce non-linearity to neural networks, allowing them to learn complex patterns in data.
5. keras.layers.Dropout: Dropout is a regularization technique used in neural networks to prevent overfitting by randomly setting a fraction of input units to zero during training.
6. keras.optimizers.SGD: SGD stands for Stochastic Gradient Descent, which is an optimization algorithm commonly used for training neural networks.
7. random: The random module provides functions for generating random numbers and choosing random items from sequences.
8. nltk: NLTK (Natural Language Toolkit) is a library for natural language processing, providing tools for tokenization, stemming, tagging, parsing, and more.
9. nltk.stem.WordNetLemmatizer: WordNetLemmatizer is a class in NLTK used for lemmatization, which is the process of reducing words to their base or root form.
10. json: The json module provides functions for encoding and decoding JSON data, which is commonly used for storing and exchanging data.
11. pickle: The pickle module implements binary protocols for serializing and deserializing Python objects, allowing objects to be saved to a file and later restored.

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.models import load_model
import random

import nltk
from nltk.stem import WordNetLemmatizer

import json
import pickle

import tkinter
from tkinter import *



2024-11-11 19:06:26.424206: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Step 2. Preprocessing the Data
The model cannot take the raw data. It has to go through a lot of pre-processing for the machine to easily understand. For textual data, there are many preprocessing techniques available. The first technique is tokenizing, in which we break the sentences into words.

By observing the intents file, we can see that each tag contains a list of patterns and responses. We tokenize each pattern and add the words in a list. Also, we create a list of classes and documents to add all the intents associated with patterns.

In [2]:
# Importing the intents JSON file which contains predefined intents and responses
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

# Initialize empty lists and set of characters to ignore
words = []          # List to store all words in the patterns
classes = []        # List to store all intent tags
documents = []      # List to store tuples of words and corresponding intent tag
ignore_letters = ['!', '?', ',', '.']  # Characters to ignore

# Loop through each intent in the intents dictionary
for intent in intents['intents']:
    # Loop through each pattern in the current intent
    for pattern in intent['patterns']:
        # Tokenize each word in the pattern
        word = nltk.word_tokenize(pattern)
        words.extend(word)  # Add individual words to the 'words' list
        # Add a tuple of the tokenized words and the intent tag to the 'documents' list
        documents.append((word, intent['tag']))
        # Add the intent tag to the 'classes' list if it's not already there
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Print the 'documents' list, which contains tokenized words and intent tags
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

Another technique is Lemmatization. We can convert words into the lemma form so that we can reduce all the canonical words. For example, the words play, playing, plays, played, etc. will all be replaced with play. This way, we can reduce the number of total words in our vocabulary. So now we lemmatize each word and remove the duplicate words.

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/embaby/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Initializing the WordNetLemmatizer for stemming words
lemmatizer = WordNetLemmatizer()

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In the end, the words contain the vocabulary of our project and classes contain the total entities to classify. To save the python object in a file, we used the pickle.dump() method. These files will be helpful after the training is done and we predict the chats.

### Step 3. Create Training and Testing Data
To train the model, we will convert each input pattern into numbers. First, we will lemmatize each word of the pattern and create a list of zeroes of the same length as the total number of words. We will set value 1 to only those indexes that contain the word in the patterns. In the same way, we will create the output by setting 1 to the class input the pattern belongs to.

In [6]:
# create the training data
training = []
# create empty array for the output
output_empty = [0] * len(classes)

# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    # Flatten the bag and output_row lists before appending
    training.append([bag, output_row])

# Shuffle the training data
random.shuffle(training)

# Flatten the training data and convert it to a numpy array
training = np.array([np.concatenate((item[0], item[1])) for item in training])

# Split the training data into X (patterns) and Y (intents)
train_x = list(training[:, :len(words)])
train_y = list(training[:, len(words):])

print("Training data is created")

Training data is created


### Step 4. Training the Model
The architecture of our model will be a neural network consisting of 3 dense layers. The first layer has 128 neurons, the second one has 64 and the last layer will have the same neurons as the number of classes. The dropout layers are introduced to reduce overfitting of the model. We have used the SGD optimizer and fit the data to start the training of the model. After the training of 200 epochs is completed, we then save the trained model using the Keras model.save(“chatbot_model.h5”) function.

In [7]:
# Define the neural network model
model = Sequential()
# Add a dense layer with 128 units and 'relu' activation function
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
# Add dropout layer with dropout rate of 0.5
model.add(Dropout(0.5))
# Add another dense layer with 64 units and 'relu' activation function
model.add(Dense(64, activation='relu'))
# Add another dropout layer with dropout rate of 0.5
model.add(Dropout(0.5))
# Add a dense layer with units equal to the number of intents and 'softmax' activation function
model.add(Dense(len(train_y[0]), activation='softmax'))

# Define learning rate scheduler
def lr_scheduler(epoch):
    if epoch < 10:
        return 0.01
    else:
        return 0.001

# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1, callbacks=[LearningRateScheduler(lr_scheduler)])
model.save('chatbot_model.h5', hist)

print("model is created")


Epoch 1/200
10/10 [==============================] - 2s 7ms/step - loss: 2.1828 - accuracy: 0.1915 - lr: 0.0100
Epoch 2/200
10/10 [==============================] - 0s 16ms/step - loss: 2.1647 - accuracy: 0.0638 - lr: 0.0100
Epoch 3/200
10/10 [==============================] - 0s 8ms/step - loss: 2.1022 - accuracy: 0.1915 - lr: 0.0100
Epoch 4/200
10/10 [==============================] - 0s 8ms/step - loss: 1.9586 - accuracy: 0.3191 - lr: 0.0100
Epoch 5/200
10/10 [==============================] - 0s 9ms/step - loss: 1.8435 - accuracy: 0.4468 - lr: 0.0100
Epoch 6/200
10/10 [==============================] - 0s 8ms/step - loss: 1.7510 - accuracy: 0.5319 - lr: 0.0100
Epoch 7/200
10/10 [==============================] - 0s 9ms/step - loss: 1.5889 - accuracy: 0.4681 - lr: 0.0100
Epoch 8/200
10/10 [==============================] - 0s 6ms/step - loss: 1.3989 - accuracy: 0.6809 - lr: 0.0100
Epoch 9/200
10/10 [==============================] - 0s 16ms/step - loss: 1.2940 - accuracy: 0.6170 - l

/Users/embaby/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


model is created


### Step 5. Interacting With the Chatbot
Our model is ready to chat, so now let’s create a nice graphical user interface for our chatbot in a new file. You can name the file as gui_chatbot.py

In our GUI file, we will be using the Tkinter module to build the structure of the desktop application and then we will capture the user message and again perform some preprocessing before we input the message into our trained model.

The model will then predict the tag of the user’s message, and we will randomly select the response from the list of responses in our intents file.

Here’s the full source code for the GUI file.

In [9]:
# Initializing the WordNetLemmatizer for stemming words
#lemmatizer = WordNetLemmatizer()


model = load_model('chatbot_model.h5')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))

def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# Creating tkinter GUI
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 

        ints = predict_class(msg)
        res = getResponse(ints, intents)
        
        ChatBox.insert(END, "Bot: " + res + '\n\n')           

        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)

root = Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatBox.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
                    command= send )

#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

root.mainloop()

2024-11-11 19:10:41.283 python[36631:970386] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/embaby/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
TypeError: send() takes 0 positional arguments but 1 was given


1/1 [==============================] - 0s 291ms/step
